In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""   Model simulation with neurolib   -- Version 2.2
Last edit:  2023/05/11
Authors:    Leone, Riccardo (RL)
Notes:      - Model simulation of the phenomenological Hopf model with Neurolib
            - Release notes:
                * Changed script to work with following steps
To do:      - 
Comments:   

Sources: 
"""
# %% Initial imports

from neurolib.models.pheno_hopf import PhenoHopfModel
from neurolib.optimize.exploration import BoxSearch
from neurolib.utils import paths
from neurolib.utils.parameterSpace import ParameterSpace
import filteredPowerSpectralDensity as filtPowSpectr
from petTOAD_setup import *


# Define the evaluation function for the model
def evaluate(traj):
    model = search.getModelFromTraj(traj)
    bold_list = []

    model.randomICs()
    model.run(chunkwise=True, chunksize=60000, append=True)
    # skip the first 120 secs to "warm up" the timeseries, 3000 msec(==TR) / 10 (==sampling freq) = 300 steps
    ts = model.outputs.x[:, 12000::300]
    t = model.outputs.t[12000::300]
    ts_filt = BOLDFilters.BandPassFilter(ts)
    bold_list.append(ts_filt)

    result_dict = {}
    result_dict["BOLD"] = np.array(bold_list).squeeze()
    result_dict["t"] = t

    search.saveToPypet(result_dict, traj)


# Choose the group on which to perform analyses ("CN_no_ WMH", "CN_WMH", "MCI_noWMH", "MCI_WMH")
group_name = 'CN_no_WMH_lqt'
group_list = CN_no_WMH

# Create the results dir for the group
SIM_DIR_GROUP = SIM_DIR / group_name
if not Path.exists(SIM_DIR_GROUP):
    Path.mkdir(SIM_DIR_GROUP)

# Set the directory where to save results
paths.HDF_DIR = str(SIM_DIR_GROUP)

# Get the timeseries for the chosen group
group, timeseries = get_group_ts_for_freqs(group_list, all_fMRI_clean)
# Get the mean frequencies (narrow bandwidth) for the chosen group and set the model frequencies
f_diff = filtPowSpectr.filtPowSpetraMultipleSubjects(timeseries, TR)
f_diff[np.where(f_diff == 0)] = np.mean(f_diff[np.where(f_diff != 0)])

delay = False
if not delay:
    Dmat_dummy = np.zeros_like(sc)
    Dmat = Dmat_dummy
else:
    pass

sc_pre = pd.read_csv(UTL_DIR / "connectivity.csv", index_col = 0).values
sc = sc_pre / sc_pre.max()
# Initialize the model (neurolib wants a Dmat to initialize the model,
# so we gave it an empty Dmat, which we also later cancel by setting it to None after the initialization of the model)
model = PhenoHopfModel(Cmat=sc, Dmat=Dmat)
# Empirical fmri is 193 timepoints with a TR=3s --> 9.65 min. We simulate 9.65 min + 2 min of initial warm up of the timeseries --> 11.65
model.params["duration"] = 11.65 * 60 * 1000
model.params["signalV"] = 0
model.params["dt"] = 0.1
model.params["sampling_dt"] = 10.0
model.params["sigma"] = 0.02
model.params["w"] = 2 * np.pi * f_diff
model.params["a"] = np.ones(n_nodes) * (-0.02)
# Define the parameters space to explore for G (K_gl)
parameters = ParameterSpace(
    {"K_gl": np.round(np.linspace(0.0, 1.5, 16), 2)}, kind="grid"
)

#%%
filename = "finding_best_G.hdf"
if __name__ == "__main__":
    n_sim = len(group)
    for i in range(n_sim):
        print(f"Now performing simulation number {i+1}/{n_sim}...")
        # Initialize the search
        search = BoxSearch(
            model=model,
            evalFunction=evaluate,
            parameterSpace=parameters,
            filename=filename,
        )
        search.run(chunkwise=True, chunksize=60000, append=True)
    print(f"Done with simulations for {group_name}")
# %%


MainProcess root INFO     Phenomenological hopf: Model initialized.
MainProcess root INFO     Number of processes: 288
MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `/home/leoner/petTOAD/results/final_simulations/CN_no_WMH_lqt/finding_best_G.hdf`.
MainProcess pypet.environment.Environment INFO     Environment initialized.
MainProcess root INFO     Number of parameter configurations: 16
MainProcess root INFO     BoxSearch: Environment initialized.
MainProcess pypet.environment.Environment INFO     I am preparing the Trajectory for the experiment and initialise the store.
MainProcess pypet.environment.Environment INFO     Initialising the storage for the trajectory.
MainProcess pypet.storageservice.HDF5StorageService INFO     Initialising storage or updating meta data of Trajectory `results-2023-11-10-10H-00M-53S`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Finished init or meta data update for `results-2023-11-10-10H-00M-53S`.
Ma

filtPowSpectraMultipleSubjects: subject 1 (of 63)
filtPowSpectraMultipleSubjects: subject 2 (of 63)
filtPowSpectraMultipleSubjects: subject 3 (of 63)
filtPowSpectraMultipleSubjects: subject 4 (of 63)
filtPowSpectraMultipleSubjects: subject 5 (of 63)
filtPowSpectraMultipleSubjects: subject 6 (of 63)
filtPowSpectraMultipleSubjects: subject 7 (of 63)
filtPowSpectraMultipleSubjects: subject 8 (of 63)
filtPowSpectraMultipleSubjects: subject 9 (of 63)
filtPowSpectraMultipleSubjects: subject 10 (of 63)
filtPowSpectraMultipleSubjects: subject 11 (of 63)
filtPowSpectraMultipleSubjects: subject 12 (of 63)
filtPowSpectraMultipleSubjects: subject 13 (of 63)
filtPowSpectraMultipleSubjects: subject 14 (of 63)
filtPowSpectraMultipleSubjects: subject 15 (of 63)
filtPowSpectraMultipleSubjects: subject 16 (of 63)
filtPowSpectraMultipleSubjects: subject 17 (of 63)
filtPowSpectraMultipleSubjects: subject 18 (of 63)
filtPowSpectraMultipleSubjects: subject 19 (of 63)
filtPowSpectraMultipleSubjects: subject 

MainProcess pypet.storageservice.HDF5StorageService INFO     Finished init or meta data update for `results-2023-11-10-10H-00M-53S`.
MainProcess pypet.environment.Environment INFO     Starting multiprocessing with at most 288 processes running at the same time.
MainProcess pypet INFO     PROGRESS: Finished  0/16 runs [                    ]  0.0%


KeyboardInterrupt: 

2023-11-10 10:01:27,443 pypet ERROR    ERROR occurred during a single run! 
Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/numba/core/serialize.py", line 30, in _numba_unpickle
    def _numba_unpickle(address, bytedata, hashed):
KeyboardInterrupt

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/pypet/environment.py", line 259, in _sigint_handling_single_run
    return _single_run(kwargs)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/pypet/environment.py", line 314, in _sing

...

  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/pypet/environment.py", line 132, in _process_single_run
    result = _sigint_handling_single_run(kwargs)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/pypet/environment.py", line 259, in _sigint_handling_single_run
    return _single_run(kwargs)
Process Process-17:
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/pypet/environment.py", line 314, in _single_run
    result = runfunc(traj, *runargs, **kwrunparams)
  File "/tmp/ipykernel_699572/1263840564.py", line 30, in evaluate
    model.run(chunkwise=True, chunksize=60000, append=True)
  File "/home/leoner/miniconda3/envs/neurolib/

  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/pypet/environment.py", line 259, in _sigint_handling_single_run
    return _single_run(kwargs)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/pypet/environment.py", line 314, in _single_run
    result = runfunc(traj, *runargs, **kwrunparams)
  File "/tmp/ipykernel_699572/1263840564.py", line 30, in evaluate
    model.run(chunkwise=True, chunksize=60000, append=True)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/neurolib/models/model.py", line 241, in run
    self.integrateChunkwise(chunksize=chunksize, bold=bold, append_outputs=append)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/neurolib/models/model.py", line 300, in integrateChunkwise
    self.autochunk(chunksize=currentChunkSize, append_outputs=append_outputs, bold=bold)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/neurolib/models/model.py", l

Process Process-18:
Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/numba/core/serialize.py", line 30, in _numba_unpickle
    def _numba_unpickle(address, bytedata, hashed):
KeyboardInterrupt

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10

Process Process-26:
Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/numba/core/serialize.py", line 30, in _numba_unpickle
    def _numba_unpickle(address, bytedata, hashed):
KeyboardInterrupt

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10

Process Process-31:
Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/site-packages/numba/core/serialize.py", line 30, in _numba_unpickle
    def _numba_unpickle(address, bytedata, hashed):
KeyboardInterrupt

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

SystemError: _PyEval_EvalFrameDefault returned a result with an exception set

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/leoner/miniconda3/envs/neurolib/lib/python3.10